# Save the precropped dataset info

In [ ]:
import os
import numpy as np
import edt

from utils import save_obj, load_obj

## 1. Specify the precropped dataset path

In [ ]:
precropped_trainset_path = "./ATM22_train/"
precropped_labelset_path = "./ATM22_label/"
precropped_validateset_path = "./ATM22_validate/"

In [ ]:
img_case_name_list = os.listdir(precropped_trainset_path)
img_case_name_list.sort()
# img_case_name_list

In [ ]:
label_case_name_list = os.listdir(precropped_labelset_path)
label_case_name_list.sort()
# label_case_name_list

In [ ]:
# Make some necessary checks
assert len(img_case_name_list) == len(label_case_name_list)

for index, img_case_name in enumerate(img_case_name_list):
    assert img_case_name == label_case_name_list[index]

#-----------------------------------------------------------------------------------

one_cropped_img_name = precropped_trainset_path + img_case_name_list[100]
print(one_cropped_img_name)
one_cropped_img = np.load(one_cropped_img_name)
one_cropped_img

In [ ]:
img_crop_cube_size = one_cropped_img.shape
img_crop_cube_size

In [ ]:
one_cropped_label_name = precropped_labelset_path + label_case_name_list[100]
one_cropped_label_name

one_cropped_label = np.load(one_cropped_label_name)
one_cropped_label

In [ ]:
label_crop_cube_size = one_cropped_label.shape
print(label_crop_cube_size)

assert img_crop_cube_size == label_crop_cube_size

## 2. Match up image with label for trainset

In [ ]:
train_dataset_dict = dict()

for index, filename in enumerate(img_case_name_list):
    print("Process {0} | {1}/{2}".format(filename, index+1, len(img_case_name_list)))
    
    case_name = filename.split(".")[0]
    # print(case_name)
    train_dataset_dict[case_name] = {}
    train_dataset_dict[case_name]['image'] = precropped_trainset_path + filename
    train_dataset_dict[case_name]['label'] = precropped_labelset_path + filename
    
    label_temp = np.load(precropped_labelset_path + filename)
    train_dataset_dict[case_name]['airway_pixel_num'] = np.sum(label_temp)
    
    label_temp = np.array(label_temp, dtype=np.uint32, order='F')
    label_temp_edt = edt.edt(label_temp, black_border=True, order='F', parallel=1)
    
    train_dataset_dict[case_name]['airway_pixel_num_boundary'] = len(np.where(label_temp_edt==1)[0])
    train_dataset_dict[case_name]['airway_pixel_num_inner'] = len(np.where(label_temp_edt>1)[0])


In [ ]:
len(train_dataset_dict.items())

In [ ]:
train_dataset_dict

## 3. Save the `train_dataset_dict` into a .pkl file

In [ ]:
dataset_info_path = "./dataset_info/"
if not os.path.exists(dataset_info_path):
    os.mkdir(dataset_info_path)

train_dataset_info_file = "{0}train_dataset_info_ATM22_crops_{1}".format(dataset_info_path, 
                                                                         img_crop_cube_size[0])

save_obj(train_dataset_dict, train_dataset_info_file)

## 4. Load the dataset_info file to check

In [ ]:
dataset_info_text = load_obj(train_dataset_info_file)

print(type(dataset_info_text))
# dataset_info_text

# Prepare the `validate_dataset_dict`

In [ ]:
validate_dataset_dict = dict()

validateset_img_case_name_list = os.listdir(precropped_validateset_path)
validateset_img_case_name_list.sort()
# validateset_img_case_name_list

for index, filename in enumerate(validateset_img_case_name_list):
    # print("Process {0} | {1}/{2}".format(filename, 
    #                                      index+1, 
    #                                      len(validateset_img_case_name_list)))
    case_name = filename.split(".")[0]
    validate_dataset_dict[case_name] = {}
    validate_dataset_dict[case_name]['image'] = precropped_validateset_path + filename

In [ ]:
len(validate_dataset_dict.items())

In [ ]:
# validate_dataset_dict

## Save the `validate_dataset_dict` into a .pkl file

In [ ]:
validate_dataset_info_file = "{0}validate_dataset_info_ATM22_crops_{1}".format(dataset_info_path, 
                                                                               img_crop_cube_size[0])
save_obj(validate_dataset_dict, validate_dataset_info_file)